In [ ]:
# !pip install pymongo langchain-perplexity
# !pip install python-dotenv



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()  # This will read the .env file and set environment variables

PPLX_API_KEY = os.environ.get("PPLX_API_KEY")
if not PPLX_API_KEY:
    raise ValueError("PPLX_API_KEY not found in environment. Check your .env file.")

In [ ]:
import os
import getpass
from pymongo import MongoClient
from typing import Dict
from datetime import datetime

# Perplexity API key setup
if "PPLX_API_KEY" not in os.environ:
    os.environ["PPLX_API_KEY"] = getpass.getpass("Enter your Perplexity API key: ")


# MongoDB connection (replace with your details)
MONGO_URI = "mongodb+srv://<username>:<password>@cluster.mongodb.net"
DB_NAME = "erp_db"
client = MongoClient(MONGO_URI)
db = client[DB_NAME]

In [ ]:
def authenticate_user(user_id: str, api_key: str) -> bool:
    """Authenticate the user against the ERP user database."""
    user = db.users.find_one({"user_id": user_id, "api_key": api_key})
    return user is not None

def get_most_profitable_customer(subscriber_id: str) -> Dict:
    """Find the most profitable customer for the given subscriber."""
    pipeline = [
        {"$match": {"subscriber_id": subscriber_id}},
        {"$group": {"_id": "$customer_id", "total_profit": {"$sum": "$profit"}}},
        {"$sort": {"total_profit": -1}},
        {"$limit": 1}
    ]
    result = list(db.sales.aggregate(pipeline))
    return result[0] if result else {}

def employees_called_in_sick(subscriber_id: str, month: int, year: int) -> int:
    """Return count of employees who called in sick in the specified month."""
    start_date = datetime(year, month, 1)
    if month == 12:
        end_date = datetime(year + 1, 1, 1)
    else:
        end_date = datetime(year, month + 1, 1)
    query = {
        "subscriber_id": subscriber_id,
        "absence_type": "sick",
        "date": {
            "$gte": start_date,
            "$lt": end_date
        }
    }
    return db.absences.count_documents(query)

In [ ]:
from langchain_perplexity import ChatPerplexity
from langchain_core.prompts import ChatPromptTemplate

chat = ChatPerplexity(temperature=0, model="sonar")  # 'sonar' or any available Perplexity model
system_msg = "You are a helpful ERP analytics assistant. Answer the user's question using the provided business data context."
prompt = ChatPromptTemplate.from_messages([
    ("system", system_msg),
    ("human", "{input}")
])
chain = prompt | chat

In [ ]:
def ask_llm_with_context(question, data_context):
    input_text = f"{question}\nRelevant business data:\n{data_context}"
    response = chain.invoke({"input": input_text})
    return response.content

In [ ]:
def chatbot_loop():
    subscriber_id = input("Enter your subscriber ID: ")
    user_id = input("Enter your user ID: ")
    api_key = getpass.getpass("API Key: ")

    if not authenticate_user(user_id, api_key):
        print("Authentication failed.")
        return

    print("Ask your question (e.g. 'Who is my most profitable customer?', 'How many called in sick in May 2025?') Type 'exit' to quit.")

    while True:
        question = input("> ")
        if question.lower() == "exit":
            break
        # Match logic can be expanded with NLP or custom regex
        if "profitable customer" in question:
            result = get_most_profitable_customer(subscriber_id)
            context = f"Most profitable customer: {result.get('_id', 'N/A')}, Total profit: {result.get('total_profit', 0)}"
            answer = ask_llm_with_context(question, context)
            print(answer)
        elif "called in sick" in question:
            # parse month/year from question for demo, e.g., "May 2025"
            parts = question.lower().split()
            month_name = next((x for x in parts if x.capitalize() in ["January","February","March","April","May","June","July","August","September","October","November","December"]), None)
            year_val = next((x for x in parts if x.isdigit() and len(x)==4), None)
            month_lookup = {"january":1,"february":2,"march":3,"april":4,"may":5,"june":6,"july":7,"august":8,"september":9,"october":10,"november":11,"december":12}
            if month_name and year_val:
                month = month_lookup[month_name]
                year = int(year_val)
                count = employees_called_in_sick(subscriber_id, month, year)
                context = f"Employees called in sick in {month_name.capitalize()} {year}: {count}"
                answer = ask_llm_with_context(question, context)
                print(answer)
            else:
                print("Please specify a valid month and year.")
        else:
            print("Sorry, I can't answer that yet (custom logic needed).")

In [ ]:
chatbot_loop()